In [ ]:
from chat_GPT import Chat
import pandas as pd
from tqdm.auto import tqdm

Importing the test data

In [ ]:
data = pd.read_csv("../data/test/clean_test.csv")
data.head(10)

,id,comment_text,label
0,0001ea8717f6de06,Thank you for understanding I think very highl...,0
1,000247e83dcc1211,Dear god this site is horrible,0
2,0002f87b16116a7f,Somebody will invariably try to add Religion ...,0
3,0003e1cccfd5a40a,It says it right there that it IS a type T...,0
4,00059ace3e3e9a53,Before adding a new product to the list m...,0
5,000663aff0fffc80,this other one from,0
6,000689dd34e20979,Reason for banning throwing This article ...,0
7,000844b52dee5f3f,blocked from editing Wikipedia,0
8,00091c35fa9d0465,Arabs are committing genocide in Iraq but no ...,1
9,000968ce11f5ee34,Please stop If you continue to vandalize Wikip...,0


Creating a subset of the data to test the GPT-3 model

In [ ]:
messages = data['comment_text'][0:1000].to_list()

## Classify the messages using GPT-3.5 turbo

Importing the chat_GPT class and creating a chat object to classify the messages. We used chunks of 10 messages to reduce the number tokens per request to the API.

In [ ]:
responses = []
chat = Chat()

for i in tqdm(range(0, len(messages), 10)):
    chunk = messages[i:i+10]
    responses += chat.batch_create_chats(chunk)

The process has been interrupted by the api after 800 messages.

Now save the responses to a local csv file.

In [ ]:
data = data[0:800]
data.loc[:, 'gpt_label'] = responses
data['gpt_label'] = data['gpt_label'].astype(int)
data.to_csv("../data/test/800_test_labeled_gpt.csv", index=False)

Calculate the F1 Score for the GPT-3 labels

In [4]:
drive.mount('/content/drive')
base_path = '/content/drive/My Drive/'
test_data_path = os.path.join(base_path, 'NLP/hate_speech_detection_pipeline/data/test/sampled_test_data_labeled_result_zeroshot.csv')

data = pd.read_csv(test_data_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
data

,id,comment_text,label,gpt_label_zeroshot
0,60aa9da08d0ea205,Plug By way of deception thou shalt do war...,1,1
1,f6392472896ce852,Remove some idiots mentions of snails and berets,1,1
2,b8030e7b908008e5,your all fucking wankers and i want to shag yo...,1,1
3,3f1f27d9e59d9356,i love to have sex with farm animals,1,1
4,b07c323945f4d5ce,Im very sorry and I was very stupid I shouldve...,1,0
...,...,...,...,...
995,e9a40900345e464b,Fuck you ill cut you ass up,1,1
996,025d45d14a2e030a,FUCK YOU YOU LIKE ANGELIQUE CARRINGTON AND TAM...,1,1
997,b8247805b384b3bf,This Sucks The entire Jules and Vincent Esc...,1,1
998,bbd29f2464db25e7,hey kendricks from pittsburgh not compton...,1,1


In [8]:
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score

def get_scores(data):
    accuracy = accuracy_score(data['label'], data['gpt_label_zeroshot'])
    # Setze zero_division=0 für Recall
    recall = recall_score(data['label'], data['gpt_label_zeroshot'], average='macro', zero_division=0)
    precision = precision_score(data['label'], data['gpt_label_zeroshot'], average='macro')
    f1 = f1_score(data['label'], data['gpt_label_zeroshot'], average='macro')
    return accuracy, recall, precision, f1

get_scores(data)

(0.842, 0.5602845110366751, 0.5678676917910223, 0.5613416234161127)

The F1Score is 0.82 which is a relatively good score. Now we'll try to outperform this score using a our BERT transformer model.

## Evaluating fine-tuned distilBERT

Evaluating function

In [1]:
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch
from torch.utils.data import TensorDataset, DataLoader
import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

def evaluate_model(model_path, test_data_path):
    """
    Evaluates a DistilBERT model for sequence classification using the specified test data path.

    Args:
    model_path (str): Path to the model checkpoint file.
    test_data_path (str): Path to the test data CSV file.
    device (str, optional): Device to evaluate the model on. Defaults to 'cpu'.

    Returns:
    tuple: The macro-averaged accuracy, recall, precision, and F1 score of the model predictions.
    """
    # Load model
    model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.load_state_dict(torch.load(model_path, map_location=torch.device(device)))
    model.to(device)
    model.eval()

    # Load and prepare test data
    df = pd.read_csv(test_data_path)
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    tokenized_data = tokenizer(list(df['comment_text']), padding=True, truncation=True, return_tensors="pt")
    labels = torch.tensor(df['label'].values)
    test_dataset = TensorDataset(tokenized_data['input_ids'], tokenized_data['attention_mask'], labels)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    # Evaluate model
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Evaluating"):
            inputs, masks, labels = batch
            inputs, masks, labels = inputs.to(device), masks.to(device), labels.to(device)

            outputs = model(inputs, attention_mask=masks)
            _, predicted = torch.max(outputs.logits, dim=1)

            all_preds.extend(predicted.tolist())
            all_labels.extend(labels.tolist())

    # Calculate macro-averaged metrics
    accuracy = accuracy_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds, average='macro')
    precision = precision_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')

    return accuracy, recall, precision, f1

Testing our Finetuned Model

In [2]:
import os
from google.colab import drive
drive.mount('/content/drive')
base_path = '/content/drive/My Drive/'
test_data_path = os.path.join(base_path, 'NLP/hate_speech_detection_pipeline/data/test/sampled_test_data_labeled_result_zeroshot.csv')
model_path = os.path.join(base_path, 'NLP/hate_speech_detection_pipeline/model/model_distil.pth')

# Calculate metrics and output them
accuracy, recall, precision, f1 = evaluate_model(model_path, test_data_path)

print("\n")
print(f'Accuracy: {accuracy}')
print(f'Recall: {recall}')
print(f'Precision: {precision}')
print(f'F1 Score: {f1}')

Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Evaluating:   0%|          | 0/16 [00:00<?, ?it/s]



Accuracy: 0.908
Recall: 0.9061945981855217
Precision: 0.9199069994497744
F1 Score: 0.9070327689279261
